In [1]:
import duckdb

In [2]:
duckdb.sql("DROP TABLE IF EXISTS employees")
duckdb.sql("CREATE TABLE employees AS SELECT * FROM 'Employee.csv'")

# Basic Exploration

In [3]:
duckdb.sql("""
          SELECT * FROM employees LIMIT 5
           """)

┌─────────────┬──────────────────┬─────────┬───────┬────────────┬────────────────────────────┬──────────────────┬─────────────────┬───────────────────┬────────────────┬─────────────────────┬──────────────────┬────────────────┬───────────┬───────────────────────┬───────────┬────────────────┬────────────┬─────────────────────────────┬──────────┐
│ Employee_ID │    Department    │ Gender  │  Age  │ Job_Title  │         Hire_Date          │ Years_At_Company │ Education_Level │ Performance_Score │ Monthly_Salary │ Work_Hours_Per_Week │ Projects_Handled │ Overtime_Hours │ Sick_Days │ Remote_Work_Frequency │ Team_Size │ Training_Hours │ Promotions │ Employee_Satisfaction_Score │ Resigned │
│    int64    │     varchar      │ varchar │ int64 │  varchar   │         timestamp          │      int64       │     varchar     │       int64       │     double     │        int64        │      int64       │     int64      │   int64   │         int64         │   int64   │     int64      │   int64    │    

In [4]:
duckdb.sql("""
          DESCRIBE employees 
           """)

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Employee_ID                 │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Department                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Gender                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Age                         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Job_Title                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Hire_Date                   │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ Years_At_Company            │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Education_Level             │ VARCHAR     │ YES     

# Salary and Compensation Insights

## Avg_Salary by Department, Job_Title

In [ ]:
# Getting Min, Max, and Avg Salary by Department and Job_Title

duckdb.sql("""
         SELECT 
            Department,
            Job_Title,
            MIN(Monthly_Salary) AS Min_Salary,
            MAX(Monthly_Salary) AS Max_Salary,
            ROUND(AVG(Monthly_Salary),2) AS Avg_Salary,
            Count(Employee_ID) AS Number_of_Employees
         FROM employees
         GROUP BY
            Department, Job_Title
         ORDER BY 
            Department, Avg_Salary DESC
          """)

┌──────────────────┬────────────┬────────────┬────────────┬────────────┬─────────────────────┐
│    Department    │ Job_Title  │ Min_Salary │ Max_Salary │ Avg_Salary │ Number_of_Employees │
│     varchar      │  varchar   │   double   │   double   │   double   │        int64        │
├──────────────────┼────────────┼────────────┼────────────┼────────────┼─────────────────────┤
│ Customer Support │ Engineer   │     6600.0 │     9000.0 │    7794.65 │                1570 │
│ Customer Support │ Manager    │     6600.0 │     9000.0 │    7792.43 │                1585 │
│ Customer Support │ Consultant │     6050.0 │     8250.0 │    7151.77 │                1558 │
│ Customer Support │ Developer  │     5500.0 │     7500.0 │     6517.9 │                1620 │
│ Customer Support │ Specialist │     4950.0 │     6750.0 │    5872.91 │                1630 │
│ Customer Support │ Analyst    │     4400.0 │     6000.0 │    5192.92 │                1582 │
│ Customer Support │ Technician │     3850.0 │    

## Identifying High-Salary Outliers: A Departmental and Job-Level Analysis Using Standard Deviation Thresholds

In [ ]:
duckdb.sql("""
WITH Salary_Stats AS (
    SELECT 
        Employee_ID,
        Department,
        Job_Title,
        Monthly_Salary * 12 AS Annual_Salary,
        ROUND(AVG(Monthly_Salary * 12) OVER (PARTITION BY Department, Job_Title), 2) AS Avg_Annual_Salary,
        ROUND(STDDEV(Monthly_Salary * 12) OVER (PARTITION BY Department, Job_Title), 2) AS Salary_StdDev
    FROM employees
),
Salary_Variance AS (
    SELECT
        Department,
        Job_Title,
        Employee_ID,
        Annual_Salary,
        Avg_Annual_Salary,
        Salary_StdDev,
        ROUND((Annual_Salary - Avg_Annual_Salary) / Salary_StdDev, 2) AS Std_Dev_Multiple
    FROM Salary_Stats
    WHERE Salary_StdDev > 0 -- Avoid division by zero
)
SELECT 
    Department,
    Job_Title,
    COUNT(Employee_ID) AS Number_Of_Employees,
    MAX(Std_Dev_Multiple) AS Max_Std_Dev_Multiple,
    Annual_Salary,
    Avg_Annual_Salary,
    ROUND(Annual_Salary - Avg_Annual_Salary, 2) As Salary_Difference
FROM Salary_Variance
GROUP BY Department, Job_Title, Annual_Salary, Avg_Annual_Salary
ORDER BY Max_Std_Dev_Multiple DESC, Salary_Difference DESC
LIMIT 10;
""")



┌──────────────────┬────────────┬─────────────────────┬──────────────────────┬───────────────┬───────────────────┬───────────────────┐
│    Department    │ Job_Title  │ Number_Of_Employees │ Max_Std_Dev_Multiple │ Annual_Salary │ Avg_Annual_Salary │ Salary_Difference │
│     varchar      │  varchar   │        int64        │        double        │    double     │      double       │      double       │
├──────────────────┼────────────┼─────────────────────┼──────────────────────┼───────────────┼───────────────────┼───────────────────┤
│ Marketing        │ Developer  │                 249 │                  1.5 │       90000.0 │           77488.1 │           12511.9 │
│ Finance          │ Engineer   │                 268 │                 1.49 │      108000.0 │          93028.06 │          14971.94 │
│ Customer Support │ Technician │                 279 │                 1.48 │       63000.0 │          54289.88 │           8710.12 │
│ Legal            │ Analyst    │                 299 │

## By Gender

┌──────────────────┬────────────┬─────────┬────────────┐
│    Department    │ Job_Title  │ Gender  │ Avg_Salary │
│     varchar      │  varchar   │ varchar │   double   │
├──────────────────┼────────────┼─────────┼────────────┤
│ Customer Support │ Analyst    │ Male    │    5226.77 │
│ Customer Support │ Consultant │ Other   │    7344.79 │
│ Customer Support │ Developer  │ Male    │    6523.13 │
│ Customer Support │ Engineer   │ Other   │    7821.82 │
│ Customer Support │ Manager    │ Female  │    7829.92 │
│ Customer Support │ Specialist │ Other   │    5964.55 │
│ Customer Support │ Technician │ Male    │    4531.35 │
│ Engineering      │ Analyst    │ Female  │    5205.38 │
│ Engineering      │ Consultant │ Female  │    7120.82 │
│ Engineering      │ Developer  │ Other   │     6561.4 │
│     ·            │    ·       │  ·      │        ·   │
│     ·            │    ·       │  ·      │        ·   │
│     ·            │    ·       │  ·      │        ·   │
│ Operations       │ Manager   

## High School Graduates Earning Top Salaries: Identifying Departments and Job Titles Where High School Education Leads the Pay Scale

In [61]:
duckdb.sql("""
WITH Avg_Salary_by_Education AS (
    SELECT 
        Department,
        Job_Title,
        Education_Level,
        COUNT(Employee_ID) AS Number_Of_Employees,
        ROUND(AVG(Monthly_Salary * 12), 2) AS Avg_Annual_Salary,
        ROW_NUMBER() OVER (PARTITION BY Department, Job_Title ORDER BY AVG(Monthly_Salary * 12) DESC) AS Rank
    FROM employees
    GROUP BY Department, Job_Title, Education_Level
)
SELECT 
    Department,
    Job_Title,
    Education_Level,
    Number_Of_Employees,
    Avg_Annual_Salary,
    RANK
FROM Avg_Salary_by_Education
WHERE (Education_Level = 'High School') AND (RANK = 1)
ORDER BY Department, Job_Title, Rank DESC;
""")


┌─────────────┬────────────┬─────────────────┬─────────────────────┬───────────────────┬───────┐
│ Department  │ Job_Title  │ Education_Level │ Number_Of_Employees │ Avg_Annual_Salary │ Rank  │
│   varchar   │  varchar   │     varchar     │        int64        │      double       │ int64 │
├─────────────┼────────────┼─────────────────┼─────────────────────┼───────────────────┼───────┤
│ Engineering │ Manager    │ High School     │                 444 │          94605.41 │     1 │
│ Engineering │ Technician │ High School     │                 462 │          55390.91 │     1 │
│ HR          │ Developer  │ High School     │                 486 │          78172.84 │     1 │
│ HR          │ Specialist │ High School     │                 466 │          70559.23 │     1 │
│ IT          │ Developer  │ High School     │                 483 │          77975.16 │     1 │
│ Legal       │ Manager    │ High School     │                 492 │          93848.78 │     1 │
│ Marketing   │ Consultant │ H

Managers can achieve the highest salaries in their departments even with only a High School degree. In the IT department, Developers with a High School degree can earn notably high salaries.

# Work-Life Balance Metrics

Before Analysis, let's get some max and min for some key fields

In [ ]:
duckdb.sql("""
            SELECT 
               max(Work_Hours_Per_Week) as Max_Work,
               min(Work_Hours_Per_Week) as Min_Work,
               max(Overtime_Hours) As max_over,
               min(Overtime_Hours) As min_over,
               max(Sick_Days) as Max_Sick_Days,
               min(Sick_Days) as min_sickdays,
               max(Remote_Work_Frequency) as max_remote,
               min(Remote_Work_Frequency) as min_remote
            FROM employees
           
          """)

┌──────────┬──────────┬──────────┬──────────┬───────────────┬──────────────┬────────────┬────────────┐
│ Max_Work │ Min_Work │ max_over │ min_over │ Max_Sick_Days │ min_sickdays │ max_remote │ min_remote │
│  int64   │  int64   │  int64   │  int64   │     int64     │    int64     │   int64    │   int64    │
├──────────┼──────────┼──────────┼──────────┼───────────────┼──────────────┼────────────┼────────────┤
│       60 │       30 │       29 │        0 │            14 │            0 │        100 │          0 │
└──────────┴──────────┴──────────┴──────────┴───────────────┴──────────────┴────────────┴────────────┘

Looks like longer working hour can decrease employees satisfaction

## Is there a relationship between remote work frequency and sick days taken?


In [94]:
duckdb.sql("""
    WITH RankedSickDays AS (
        SELECT
            Department,
            Remote_Work_Frequency,
            COUNT(Employee_ID) AS Number_of_Employees,
            ROUND(AVG(Sick_Days), 3) AS Avg_Sick_Days,
            ROW_NUMBER() OVER (PARTITION BY Department ORDER BY AVG(Sick_Days) DESC) AS RANK
        FROM employees
        GROUP BY Department, Remote_Work_Frequency
    )
    SELECT
        Department,
        Remote_Work_Frequency,
        Number_of_Employees,
        Avg_Sick_Days,
        RANK
    FROM RankedSickDays
    WHERE RANK = 1 OR RANK = 5
    ORDER BY RANK, Avg_Sick_Days DESC
""")


┌──────────────────┬───────────────────────┬─────────────────────┬───────────────┬───────┐
│    Department    │ Remote_Work_Frequency │ Number_of_Employees │ Avg_Sick_Days │ RANK  │
│     varchar      │         int64         │        int64        │    double     │ int64 │
├──────────────────┼───────────────────────┼─────────────────────┼───────────────┼───────┤
│ Finance          │                     0 │                2199 │         7.229 │     1 │
│ Customer Support │                   100 │                2248 │         7.184 │     1 │
│ Sales            │                    75 │                2288 │          7.18 │     1 │
│ IT               │                   100 │                2178 │         7.145 │     1 │
│ Engineering      │                    50 │                2142 │         7.102 │     1 │
│ HR               │                   100 │                2145 │          7.09 │     1 │
│ Marketing        │                     0 │                2163 │         7.083 │     1 │

Looks like there is no relationship between Remote_Work_Frequency and avg_sick_days

## How does overtime correlate with employee satisfaction? Show Overtime Situation by Department As well

In [ ]:
duckdb.sql("""
  WITH DepartmentStats AS (
    SELECT
      Department,
      Count(Employee_ID) AS Total_Employees_by_Department,
    FROM employees
    GROUP BY Department
           ),
  OvertimeStats AS (
    SELECT
      Department,
      CASE
        WHEN Overtime_Hours BETWEEN 0 AND 9 THEN '<10HOURS'
        WHEN Overtime_Hours BETWEEN 10 AND 20 THEN '10-20HOURS'
        ELSE '20+HOURS'
      END AS Overtime_category,
      ROUND(AVG(Employee_Satisfaction_Score),3) AS Avg_satisfaction,
      COUNT(Employee_ID) AS Number_Of_Employees_By_Category
    FROM employees
    GROUP BY
      Department, Overtime_category         
           )
  SELECT
    ds.Department as Department,
    os.Overtime_category as Overtime_category,
    os.Number_Of_Employees_By_Category as Number_Of_Employees_By_Category,
    ds.Total_Employees_by_Department as Total_Employees_by_Department,
    ROUND( 100 * os.Number_Of_Employees_By_Category / ds.Total_Employees_by_Department ,2) as Overtime_Percentage_by_Department,
    os.Avg_satisfaction as Avg_satisfaction_by_category
  FROM DepartmentStats ds
  JOIN OvertimeStats os
  ON ds.Department = os.Department
  ORDER BY 
           ds.Department, Avg_satisfaction DESC
""")

┌──────────────────┬───────────────────┬─────────────────────────────────┬───────────────────────────────┬───────────────────────────────────┬──────────────────────────────┐
│    Department    │ Overtime_category │ Number_Of_Employees_By_Category │ Total_Employees_by_Department │ Overtime_Percentage_by_Department │ Avg_satisfaction_by_category │
│     varchar      │      varchar      │              int64              │             int64             │              double               │            double            │
├──────────────────┼───────────────────┼─────────────────────────────────┼───────────────────────────────┼───────────────────────────────────┼──────────────────────────────┤
│ Customer Support │ <10HOURS          │                            3775 │                         11116 │                             33.96 │                        2.981 │
│ Customer Support │ 10-20HOURS        │                            4056 │                         11116 │                        

## The Percentage of People overtime in the company and the comparsion of overtime and non-overtime satisfication

In [66]:
duckdb.sql("""
  WITH OvertimeStats AS (
      SELECT 
          Department,
          SUM(CASE WHEN Overtime_Hours > 0 THEN 1 ELSE 0 END) AS Overtime_Employees,
          COUNT(Employee_ID) AS Total_Employees,
          ROUND(AVG(CASE WHEN Overtime_Hours > 0 THEN Employee_Satisfaction_Score ELSE NULL END), 3) AS Avg_satisfaction_For_OT_Employees
      FROM 
          employees
      GROUP BY
          Department
  ),
  NoOvertimeStats AS (
      SELECT
          Department,
          ROUND(AVG(Employee_Satisfaction_Score), 3) AS Avg_satisfaction_For_No_OT_Employees
      FROM 
          employees
      WHERE 
          Overtime_Hours < 1
      GROUP BY 
          Department
  )
  SELECT 
      os.Department,
      ROUND(100.0 * os.Overtime_Employees / os.Total_Employees, 2) AS Overtime_Employees_Percentage,
      os.Total_Employees,
      os.Avg_satisfaction_For_OT_Employees,
      nos.Avg_satisfaction_For_No_OT_Employees,
      CASE
           WHEN  os.Avg_satisfaction_For_OT_Employees > nos.Avg_satisfaction_For_No_OT_Employees then 'Yes'
           ELSE 'No'
    END AS Satisfication_higher_for_overtime
  FROM 
      OvertimeStats os
  LEFT JOIN 
      NoOvertimeStats nos
  ON 
      os.Department = nos.Department
  ORDER BY 
      Satisfication_higher_for_overtime DESC
""")


┌──────────────────┬───────────────────────────────┬─────────────────┬───────────────────────────────────┬──────────────────────────────────────┬───────────────────────────────────┐
│    Department    │ Overtime_Employees_Percentage │ Total_Employees │ Avg_satisfaction_For_OT_Employees │ Avg_satisfaction_For_No_OT_Employees │ Satisfication_higher_for_overtime │
│     varchar      │            double             │      int64      │              double               │                double                │              varchar              │
├──────────────────┼───────────────────────────────┼─────────────────┼───────────────────────────────────┼──────────────────────────────────────┼───────────────────────────────────┤
│ HR               │                         96.69 │           10960 │                             2.998 │                                2.905 │ Yes                               │
│ Finance          │                         96.62 │           11200 │                    

Except for Customer Support and Engineering Department, Avg Satisfication is higher for other departments if they don't need to work over time

# Employee Engagement

## How does work hours per week impact performance scores across different departments?

In [68]:
duckdb.sql("""
  SELECT
    Department,
    CASE
      WHEN Work_Hours_Per_Week <= 40 THEN 'Less than or Equal 40 Hours'
      WHEN Work_Hours_Per_Week BETWEEN 40 AND 60 THEN '40 to 60 Hours'
      WHEN Work_Hours_Per_Week > 60 THEN 'More than 60 Hours'
    END as WorkHour_Category,
    ROUND(AVG(Employee_Satisfaction_Score),3) AS Avg_satisfaction
  FROM
    employees
  GROUP BY
    Department, WorkHour_Category
  ORDER BY
    Department, Avg_satisfaction DESC
           
           """)

┌──────────────────┬─────────────────────────────┬──────────────────┐
│    Department    │      WorkHour_Category      │ Avg_satisfaction │
│     varchar      │           varchar           │      double      │
├──────────────────┼─────────────────────────────┼──────────────────┤
│ Customer Support │ Less than or Equal 40 Hours │            2.969 │
│ Customer Support │ 40 to 60 Hours              │            2.968 │
│ Engineering      │ Less than or Equal 40 Hours │            3.015 │
│ Engineering      │ 40 to 60 Hours              │            2.997 │
│ Finance          │ Less than or Equal 40 Hours │            3.019 │
│ Finance          │ 40 to 60 Hours              │            2.996 │
│ HR               │ Less than or Equal 40 Hours │            3.018 │
│ HR               │ 40 to 60 Hours              │            2.982 │
│ IT               │ Less than or Equal 40 Hours │            3.025 │
│ IT               │ 40 to 60 Hours              │            3.022 │
│ Legal            │

## How does training impact employee satisfaction and performance, group by department and training level, order by performance

In [13]:
duckdb.sql("""
        SELECT
           Department,
           CASE
            WHEN Training_Hours < 20 THEN 'LOW'
            WHEN Training_Hours BETWEEN 20 AND 40 THEN 'MEDIUM'
            ELSE 'HIGH'
           END AS Training_level,
           ROUND(AVG(Employee_Satisfaction_Score),3) AS Avg_satisfaction,
           ROUND(AVG(PerfoRmance_Score),3) AS Avg_performance
        FROM employees
        GROUP BY Department, Training_level
        ORDER BY Avg_performance DESC
         LIMIT 10
           """)

┌──────────────────┬────────────────┬──────────────────┬─────────────────┐
│    Department    │ Training_level │ Avg_satisfaction │ Avg_performance │
│     varchar      │    varchar     │      double      │     double      │
├──────────────────┼────────────────┼──────────────────┼─────────────────┤
│ HR               │ MEDIUM         │            2.965 │           3.041 │
│ Operations       │ MEDIUM         │            3.025 │           3.037 │
│ Customer Support │ LOW            │            2.954 │           3.034 │
│ Engineering      │ HIGH           │            2.999 │           3.027 │
│ Sales            │ LOW            │            3.015 │           3.023 │
│ IT               │ HIGH           │            3.036 │           3.017 │
│ Engineering      │ LOW            │            2.994 │           3.015 │
│ Customer Support │ MEDIUM         │            2.995 │           3.009 │
│ Legal            │ LOW            │            3.006 │           3.007 │
│ Engineering      │ MEDI

Doesn't look like training hours affect performace

## Comparative Analysis of Average Performance Scores and Project Handling in Marketing, Customer Support, and Sales Department

In [ ]:
duckdb.sql("""
  WITH ranked_performance AS (
  SELECT
           Department,
           Projects_Handled,
           COUNT(Employee_ID) AS Number_Of_Employees,
           ROUND(AVG(Performance_Score), 3) AS Avg_Performance_Score,
           RANK() OVER (PARTITION BY Department ORDER BY Avg_Performance_Score DESC) AS rank_most,
           RANK() OVER (PARTITION BY Department ORDER BY Avg_Performance_Score ASC) AS rank_less
  FROM     employees
  GROUP BY 
           Department, Projects_Handled
  ORDER BY 
           Department, Projects_Handled DESC
  )
  SELECT 
           Department,
           Projects_Handled,
           Number_Of_Employees,
           Avg_Performance_Score,
           CASE 
              WHEN rank_most <= 3 THEN 'Top 3 Most'
              WHEN rank_less <= 3 THEN 'Top 3 Least'
          END AS Performance_Category
  FROM ranked_performance
  WHERE 
           (rank_most <= 3 OR rank_less <= 3) AND
           Department IN ('Marketing', 'Customer Support', 'Sales')
  ORDER BY 
           Department, Avg_Performance_Score DESC, Projects_Handled DESC;
        
""")

┌──────────────────┬──────────────────┬─────────────────────┬───────────────────────┬──────────────────────┐
│    Department    │ Projects_Handled │ Number_Of_Employees │ Avg_Performance_Score │ Performance_Category │
│     varchar      │      int64       │        int64        │        double         │       varchar        │
├──────────────────┼──────────────────┼─────────────────────┼───────────────────────┼──────────────────────┤
│ Customer Support │               39 │                 185 │                 3.189 │ Top 3 Most           │
│ Customer Support │               11 │                 242 │                 3.182 │ Top 3 Most           │
│ Customer Support │               49 │                 228 │                 3.154 │ Top 3 Most           │
│ Customer Support │               15 │                 225 │                 2.853 │ Top 3 Least          │
│ Customer Support │               21 │                 214 │                 2.818 │ Top 3 Least          │
│ Customer Support 

Performace doesn't show strong relationship with number of projects they handled

## For each department, rank employees based on their performance score, and calculate the percentage difference from the department average.

In [85]:
duckdb.sql("""
  WITH department_avg AS (
  SELECT 
    Department,
    
    ROUND(AVG(Performance_Score), 3) AS avg_performance_score
  FROM employees
  GROUP BY Department
  ),
  performance_rank AS (
  SELECT
    Department,
    Performance_Score,
    COUNT(Employee_ID) AS Number_Of_Employees,
    DENSE_RANK() OVER (PARTITION BY Department ORDER BY Performance_Score DESC) AS dept_rank
  FROM employees
  GROUP BY Department, Performance_Score
  )
  SELECT    
           pr.Department,
           pr.Number_Of_Employees,
           pr.Performance_Score,
           pr.dept_rank,
           da.avg_performance_score,
           ROUND(((pr.Performance_Score - da.avg_performance_score) / da.avg_performance_score) * 100, 2) AS pct_diff_from_avg        
  FROM performance_rank pr
  JOIN department_avg da 
  ON pr.Department = da.Department
  WHERE dept_rank <= 2
  ORDER BY 
           pr.Department, pr.dept_rank
  """)

┌──────────────────┬─────────────────────┬───────────────────┬───────────┬───────────────────────┬───────────────────┐
│    Department    │ Number_Of_Employees │ Performance_Score │ dept_rank │ avg_performance_score │ pct_diff_from_avg │
│     varchar      │        int64        │       int64       │   int64   │        double         │      double       │
├──────────────────┼─────────────────────┼───────────────────┼───────────┼───────────────────────┼───────────────────┤
│ Customer Support │                2215 │                 5 │         1 │                 2.997 │             66.83 │
│ Customer Support │                2226 │                 4 │         2 │                 2.997 │             33.47 │
│ Engineering      │                2270 │                 5 │         1 │                  3.02 │             65.56 │
│ Engineering      │                2144 │                 4 │         2 │                  3.02 │             32.45 │
│ Finance          │                2182 │      

# Resignation and Retention Insights

## What factors are most strongly correlated with employee resignation?

In [9]:
duckdb.sql("""
      SELECT 
        Department,
        ROUND(AVG(CASE WHEN Resigned = TRUE THEN 1 ELSE 0 END),3) AS resignation_rate,
        ROUND(AVG(Employee_Satisfaction_Score),3) AS avg_satisfaction,
        ROUND(AVG(Performance_Score),3) AS avg_performance,
        ROUND(AVG(Work_Hours_Per_Week),3) AS avg_working_hours,
        ROUND(AVG(Overtime_Hours ),3) AS avg_overtime,
        ROUND(AVG(Monthly_Salary),3) AS avg_salary
      FROM employees
      GROUP BY Department
      ORDER BY resignation_rate DESC;
""")

┌──────────────────┬──────────────────┬──────────────────┬─────────────────┬───────────────────┬──────────────┬────────────┐
│    Department    │ resignation_rate │ avg_satisfaction │ avg_performance │ avg_working_hours │ avg_overtime │ avg_salary │
│     varchar      │      double      │      double      │     double      │      double       │    double    │   double   │
├──────────────────┼──────────────────┼──────────────────┼─────────────────┼───────────────────┼──────────────┼────────────┤
│ Finance          │            0.105 │            3.004 │           2.981 │            44.894 │       14.345 │   6398.576 │
│ HR               │            0.103 │            2.995 │           2.997 │            44.929 │       14.577 │   6400.356 │
│ Legal            │            0.102 │            2.985 │           2.982 │            45.013 │       14.548 │   6391.275 │
│ Marketing        │              0.1 │            2.994 │           2.981 │            44.888 │       14.483 │   6377.858 │


From above factors chosen, there is no strong correlation with resignation

## Is there a relationship between promotions and employee retention?

In [11]:
duckdb.sql("""
  SELECT 
           Promotions,
           COUNT(*) total_employees,
           SUM(CASE WHEN Resigned = FALSE THEN 1 ELSE 0 END) AS retained_employees,
           ROUND(SUM(CASE WHEN Resigned = FALSE THEN 1 ELSE 0 END) * 100 / count(*),2) AS retained_employees
  FROM employees
  GROUP BY Promotions
  ORDER BY Promotions
           """)

┌────────────┬─────────────────┬────────────────────┬────────────────────┐
│ Promotions │ total_employees │ retained_employees │ retained_employees │
│   int64    │      int64      │       int128       │       double       │
├────────────┼─────────────────┼────────────────────┼────────────────────┤
│          0 │           33296 │              29952 │              89.96 │
│          1 │           33436 │              30093 │               90.0 │
│          2 │           33268 │              29945 │              90.01 │
└────────────┴─────────────────┴────────────────────┴────────────────────┘

The Retained Rate is higher if got promotion

## How does tenure (years at company) affect resignation rates (group by gender)?

In [ ]:
duckdb.sql("""
  SELECT    
            CASE 
              WHEN Years_At_Company < 2 THEN '0-1 years'
              WHEN Years_At_Company BETWEEN 2 AND 5 THEN '2-5 years'
              WHEN Years_At_Company BETWEEN 6 AND 10 THEN '6-10 years'
              ELSE '10+ years'
            END AS tenure_group,
           Gender, 
            COUNT(*) AS total_employees,
            ROUND(SUM(CASE WHEN Resigned = TRUE THEN 1 ELSE 0 END) * 100 / count(*),3) AS resigned_rate
    FROM employees
    GROUP BY tenure_group, Gender
    ORDER BY tenure_group, Gender
           """)

┌──────────────┬─────────┬─────────────────┬───────────────┐
│ tenure_group │ Gender  │ total_employees │ resigned_rate │
│   varchar    │ varchar │      int64      │    double     │
├──────────────┼─────────┼─────────────────┼───────────────┤
│ 0-1 years    │ Female  │            9628 │        10.012 │
│ 0-1 years    │ Male    │            9644 │        10.027 │
│ 0-1 years    │ Other   │             829 │        12.304 │
│ 2-5 years    │ Female  │           19244 │         9.863 │
│ 2-5 years    │ Male    │           19401 │        10.061 │
│ 2-5 years    │ Other   │            1557 │          9.12 │
│ 6-10 years   │ Female  │           19129 │        10.215 │
│ 6-10 years   │ Male    │           18986 │         9.944 │
│ 6-10 years   │ Other   │            1582 │         9.039 │
└──────────────┴─────────┴─────────────────┴───────────────┘

Looks like resigned rate is different between males and female in different serving years group

## Show a Table of Number of Employees, New Hire Employees and Leaving Employees as well as the employees change rate from 2015 to 2022

In [67]:
duckdb.sql("""
WITH   EmploymentStats AS (
              SELECT 
                     Gender,
                     YEAR(Hire_Date) as Hire_year,
                     YEAR(Hire_Date) + Years_At_Company as Leave_Year
              FROM employees
       ),
       YearlyStats AS (
              SELECT 
                     y.Year as Research_Year,
                     es.Gender as Gender,
                     COUNT(*) as Total_Employees,
                     COUNT(*) FILTER (WHERE y.Year = es.Hire_Year) as New_Hires,
                     COUNT(*) FILTER (WHERE y.Year < es.Leave_Year AND y.Year > es.Hire_Year) as Old_Employees,
                     COUNT(*) FILTER (WHERE y.Year = es.Leave_Year) as Left_Employees,
              FROM
                     (SELECT DISTINCT YEAR(Hire_Date) as Year FROM employees WHERE YEAR(Hire_Date) BETWEEN 2015 AND 2023) AS y
              LEFT JOIN EmploymentStats es
                     ON y.Year BETWEEN es.Hire_year AND es.Leave_Year 
              GROUP BY
                     y.Year, es.Gender
              ORDER BY
                     y.Year
              ),
       CurrentPreYearStats AS (
              SELECT
                     Research_Year,
                     Gender,
                     Total_Employees,
                     New_Hires,
                     Old_Employees,
                     Left_Employees,
                     LAG(Total_Employees) OVER (PARTITION BY Gender ORDER BY Research_Year) AS Prev_Employees,
                     LAG(New_Hires) OVER (PARTITION BY Gender ORDER BY Research_Year) AS Prev_New_Hires,
                     LAG(Left_Employees) OVER (PARTITION BY Gender ORDER BY Research_Year) AS Prev_Employees_Leaving
              FROM YearlyStats    
              ORDER BY
                     Research_Year
              )
              SELECT
                     Research_Year,
                     Gender,
                     Total_Employees,
                     New_Hires,
                     Old_Employees,
                     Left_Employees,
                     ROUND((Total_Employees - COALESCE(Prev_Employees, Total_Employees)) / NULLIF(Prev_Employees, 0) * 100, 2) AS Total_Employees_Change_Rate,
                     ROUND((New_Hires - COALESCE(Prev_New_Hires, New_Hires)) / NULLIF(Prev_New_Hires, 0) * 100, 2) AS New_Hire_Employees_Change_Rate,
                     ROUND(New_Hires / Total_Employees * 100, 2) AS New_Hires_of_Total_Percentage,
                     ROUND((Left_Employees - COALESCE(Prev_Employees_Leaving, Left_Employees)) / NULLIF(Prev_Employees_Leaving, 0) * 100, 2) AS Employees_Leaving_Change_Rate
              FROM CurrentPreYearStats
              WHERE Gender in ('Female', 'Male')
              ORDER BY
                     Gender, Research_Year
           """)

┌───────────────┬─────────┬─────────────────┬───────────┬───────────────┬────────────────┬─────────────────────────────┬────────────────────────────────┬───────────────────────────────┬───────────────────────────────┐
│ Research_Year │ Gender  │ Total_Employees │ New_Hires │ Old_Employees │ Left_Employees │ Total_Employees_Change_Rate │ New_Hire_Employees_Change_Rate │ New_Hires_of_Total_Percentage │ Employees_Leaving_Change_Rate │
│     int64     │ varchar │      int64      │   int64   │     int64     │     int64      │           double            │             double             │            double             │            double             │
├───────────────┼─────────┼─────────────────┼───────────┼───────────────┼────────────────┼─────────────────────────────┼────────────────────────────────┼───────────────────────────────┼───────────────────────────────┤
│          2015 │ Female  │            6298 │      4807 │          1491 │              0 │                        NULL │        

For 2024, the latest data is Sep and this results in the decrease of new hire. Superisely, there is no left_employees until 2023 and new hire for both gender are similar each year

# Diversity and Inclusion Metrics

## Show the rank of the salary of age over 60+ group in their department

In [16]:
duckdb.sql("""
WITH Age_Grouped_Salary AS (
    SELECT 
        Department,
        CASE 
            WHEN Age < 20 THEN 'Less than 20'
            WHEN Age BETWEEN 20 AND 29 THEN '20 - 29'
            WHEN Age BETWEEN 30 AND 39 THEN '30 - 39'
            WHEN Age BETWEEN 40 AND 49 THEN '40 - 49'
            WHEN Age BETWEEN 50 AND 59 THEN '50 - 59'
            WHEN Age >= 60 THEN '60+'
        END AS Age_Group,
        ROUND(AVG(Monthly_Salary * 12), 2) AS Annual_Avg_Salary
    FROM employees
    GROUP BY Department, 
             CASE 
                 WHEN Age < 20 THEN 'Less than 20'
                 WHEN Age BETWEEN 20 AND 29 THEN '20 - 29'
                 WHEN Age BETWEEN 30 AND 39 THEN '30 - 39'
                 WHEN Age BETWEEN 40 AND 49 THEN '40 - 49'
                 WHEN Age BETWEEN 50 AND 59 THEN '50 - 59'
                 WHEN Age >= 60 THEN '60+'
             END
)
SELECT 
    Department,
    Age_Group,
    Annual_Avg_Salary,
    ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Annual_Avg_Salary DESC) AS Rank
FROM Age_Grouped_Salary
WHERE Age_Group = '60+'
ORDER BY Department, Rank;
""")


┌──────────────────┬───────────┬───────────────────┬───────┐
│    Department    │ Age_Group │ Annual_Avg_Salary │ Rank  │
│     varchar      │  varchar  │      double       │ int64 │
├──────────────────┼───────────┼───────────────────┼───────┤
│ Customer Support │ 60+       │          77543.82 │     1 │
│ Engineering      │ 60+       │          79131.11 │     1 │
│ Finance          │ 60+       │          77077.42 │     1 │
│ HR               │ 60+       │          76451.21 │     1 │
│ IT               │ 60+       │          76791.58 │     1 │
│ Legal            │ 60+       │          79958.96 │     1 │
│ Marketing        │ 60+       │          77002.68 │     1 │
│ Operations       │ 60+       │          77426.99 │     1 │
│ Sales            │ 60+       │           75417.3 │     1 │
└──────────────────┴───────────┴───────────────────┴───────┘

Wowwww ! People over 60 have highest salary in their department. There is no age discrimination in this company and they more focus on their experiences

## Identifying the Top-Earning Gender Across Departments and Job Titles: A Ranking-Based Analysis

In [30]:
duckdb.sql("""
         With Avg_Stats AS (
           SELECT 
               Department,
               Job_Title,
               Gender,
               ROUND(AVG(Monthly_Salary),2) AS Avg_Salary,
               ROW_NUMBER() OVER (PARTITION BY Department, Job_Title ORDER BY AVG(Monthly_Salary) DESC) AS Ranking
            FROM employees
            GROUP BY
               Department, Job_Title, Gender
            ORDER BY 
               Department, Job_Title, Gender, Avg_Salary DESC
           )
           SELECT
               Department,
               Job_Title,
               Gender,
               Avg_Salary
           FROM
               Avg_Stats
           WHERE
               RANKING = 1
           ORDER BY
                Department, Job_Title
          """)

┌──────────────────┬────────────┬─────────┬────────────┐
│    Department    │ Job_Title  │ Gender  │ Avg_Salary │
│     varchar      │  varchar   │ varchar │   double   │
├──────────────────┼────────────┼─────────┼────────────┤
│ Customer Support │ Analyst    │ Male    │    5226.77 │
│ Customer Support │ Consultant │ Other   │    7344.79 │
│ Customer Support │ Developer  │ Male    │    6523.13 │
│ Customer Support │ Engineer   │ Other   │    7821.82 │
│ Customer Support │ Manager    │ Female  │    7829.92 │
│ Customer Support │ Specialist │ Other   │    5964.55 │
│ Customer Support │ Technician │ Male    │    4531.35 │
│ Engineering      │ Analyst    │ Female  │    5205.38 │
│ Engineering      │ Consultant │ Female  │    7120.82 │
│ Engineering      │ Developer  │ Other   │     6561.4 │
│     ·            │    ·       │  ·      │        ·   │
│     ·            │    ·       │  ·      │        ·   │
│     ·            │    ·       │  ·      │        ·   │
│ Operations       │ Manager   

## To calculate the gender pay gap percentage by job title. Focusing on employees work in company over ten years and both males an females data existed

In [4]:
duckdb.sql("""
  WITH pay_stats AS (
    SELECT
        Job_Title,
           Gender,
           Years_At_Company,
           ROUND(AVG(Monthly_Salary),2) AS avg_salary,
           COUNT(*) AS employee_count
    FROM employees
    WHERE Years_At_Company
    GROUP BY Job_Title, Gender, Years_At_Company
   
           )
    SELECT
        Job_Title,
        Years_At_Company,
        MAX(CASE WHEN Gender = 'Male' THEN avg_salary END) AS male_avg_salary,
        MAX(CASE WHEN Gender = 'Female' THEN avg_salary END) AS female_avg_salary,
        ROUND((MAX(CASE WHEN Gender = 'Male' THEN avg_salary END) - MAX(CASE WHEN Gender = 'Female' THEN avg_salary END)) /
            MAX(CASE WHEN Gender = 'Male' THEN avg_salary END) * 100,2) AS gender_pay_gap_percentage
    FROM pay_stats
    GROUP BY Job_Title, Years_At_Company
    HAVING COUNT(DISTINCT Gender) = 2 AND male_avg_salary IS NOT NULL AND female_avg_salary IS NOT NULL
    ORDER BY gender_pay_gap_percentage DESC 
""")

┌────────────┬──────────────────┬─────────────────┬───────────────────┬───────────────────────────┐
│ Job_Title  │ Years_At_Company │ male_avg_salary │ female_avg_salary │ gender_pay_gap_percentage │
│  varchar   │      int64       │     double      │      double       │          double           │
├────────────┼──────────────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ Consultant │               10 │          7425.0 │            7150.0 │                       3.7 │
│ Developer  │               10 │          5625.0 │            6400.0 │                    -13.78 │
└────────────┴──────────────────┴─────────────────┴───────────────────┴───────────────────────────┘